In [ ]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

# 1. Basic autograd examples

In [ ]:
# Example1.

# Create Tensors
# 1.과 같이 .을 붙이는 이유는 텐서에 들어가는 포인트가 플롯형이어야 하기 때문입니다.
x = torch.tensor(1., requires_grad = True)
w = torch.tensor(2., requires_grad = True)
b = torch.tensor(3., requires_grad = True)

In [ ]:
# Build a Computational Graph
y = x*w + b

# Compute Gradients.
y.backward()

print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


In [ ]:
# grad는 tensor의 grad에 누적됩니다.

x = torch.tensor(1., requires_grad = True)
w = torch.tensor(2., requires_grad = True)
b = torch.tensor(3., requires_grad = True)

for _ in range(5):
    y = x*w + b
    y.backward()
    print('x\'s grad is accumulated:', x.grad)

x's grad is accumulated: tensor(2.)
x's grad is accumulated: tensor(4.)
x's grad is accumulated: tensor(6.)
x's grad is accumulated: tensor(8.)
x's grad is accumulated: tensor(10.)


In [ ]:
# Example2.

# Create Tensors of shape (10,3) and (10, 2)
x = torch.ones(10,3) # 초기값을 1로 주어 computation이 어떻게 이뤄지는지 알아봅시다.
y = torch.ones(10,2)
print('x: ', x)

# Build a fully connected layer.
linear = nn.Linear(3, 2)
print('w: ', linear.weight)
print('b: ', linear.bias)

x:  tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
w:  Parameter containing:
tensor([[ 0.3120,  0.1399, -0.4323],
        [-0.3648,  0.0290, -0.5620]], requires_grad=True)
b:  Parameter containing:
tensor([-0.4878, -0.1206], requires_grad=True)


In [ ]:
# Build loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)

In [ ]:
pred
# 10행 3열의 x에 2행 3열인 w를 transpose해서 행렬곱을 하고 bias를 더합니다.
# 실제 계산에서는 x에 합벡터를 concat하고 w와 bias를 concat해서 서로 행렬곱을 하는 식이 되겠네요.

tensor([[-0.4683, -1.0185],
        [-0.4683, -1.0185],
        [-0.4683, -1.0185],
        [-0.4683, -1.0185],
        [-0.4683, -1.0185],
        [-0.4683, -1.0185],
        [-0.4683, -1.0185],
        [-0.4683, -1.0185],
        [-0.4683, -1.0185],
        [-0.4683, -1.0185]], grad_fn=<AddmmBackward0>)

In [ ]:
# Compute Loss
loss = criterion(pred, y)
print('loss: ', loss.item())

loss:  3.1151137351989746


In [ ]:
# Backward pass.
loss.backward()

# Print out the gradients.
print('dL/dw: ', linear.weight.grad)
print('dL/db: ', linear.bias.grad)

dL/dw:  tensor([[-1.4683, -1.4683, -1.4683],
        [-2.0185, -2.0185, -2.0185]])
dL/db:  tensor([-1.4683, -2.0185])


In [ ]:
# 1-step gradient descent.
optimizer.step()

In [ ]:
# # optimizer.step()을 low level로 수행하면 아래와 같습니다.
# # sub인 이유는 최적화시 gradient에 -를 해서 내려가는 식으로 작동하기 때문입니다.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

In [ ]:
# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1-step gradient descent : ', loss.item())

loss after 1-step gradient descent :  2.8708889484405518


In [ ]:
for i in range(0, 3):
    print(f'{i} epoch:' + '='*30)

    pred = linear(x)
    loss = criterion(pred, y) # computational graph가 만들어지고
    print(f'loss after {i}-step gradient descent : ', loss.item())

    loss.backward() # 만들어진 computational graph에서 auto_grad로 계산된 grad가 linear로 전파됩니다.
    print('dL/dw: ', linear.weight.grad)
    print('dL/db: ', linear.bias.grad)
    
    optimizer.zero_grad() #해주지 않으면 그레디언트가 계속 누적되어 이상한 결과를 낸다.
    optimizer.step() # linear의 parameters()에서 전파되었던 grad로 SGD가 최적화하여 weight를 업데이트 시킵니다.

0 epoch:==============================
loss after 0-step gradient descent :  2.8708889484405518
dL/dw:  tensor([[-2.8778, -2.8778, -2.8778],
        [-3.9563, -3.9563, -3.9563]])
dL/db:  tensor([-2.8778, -3.9563])
1 epoch:==============================
loss after 1-step gradient descent :  2.421124219894409
dL/dw:  tensor([[-4.1722, -4.1722, -4.1722],
        [-5.7358, -5.7358, -5.7358]])
dL/db:  tensor([-4.1722, -5.7358])
2 epoch:==============================
loss after 2-step gradient descent :  1.8370624780654907
dL/dw:  tensor([[-5.2998, -5.2998, -5.2998],
        [-7.2859, -7.2859, -7.2859]])
dL/db:  tensor([-5.2998, -7.2859])


# 2. Loading Data from numpy

In [ ]:
# Create a numpy array.
x = np.array([[1, 2],
              [3,4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

# 3. Input Pipeline

In [ ]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transforms.ToTensor(),
                                             download=True,
                                             )

# Fetch one data pair (read data from disk)
image, label = train_dataset[0]
print(image.size())
print(label)

In [ ]:
# Data Loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=32,
                                           shuffle=True,
                                           )

In [ ]:
# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of the data loader is as below
for images, labels in train_loader:
    # Train코드가 이 자리에 들어와야 합니다.
    pass

In [ ]:
# Input pipeline for Custom Dataset

import pandas as pd
import os
from torchvision.io import read_image

to_tensor = ToTensor()

# Build Custom dataset Class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, annot_file, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_labels = pd.read_csv(annot_file, names=['file_name', 'target'])
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.img_dir)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            transform_target = self.target_transform(label)
            label = transform_target
        
        image = to_tensor(image)
        target = torch.as_tensor(label)

        return image, target

In [ ]:
## Plus
import cv2

def generate_target(target):
    pass

class CustomAdvancedDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, annot_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.imgs = list(sorted(os.listdir(self.img_dir)))
        self.annot_dir = annot_dir
        self.annots = list(sorted(os.listdir(self.annot_dir)))

        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        file_image = self.imgs[idx]
        file_target = self.annots[idx]

        img_path = os.path.join(self.img_dir, file_image)
        target_path = os.path.join(self.annot_dir, file_target)

        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        target = generate_target(target_path)

        if self.transform:
            img = self.transform(img)
        if self.target_transform:
            transform_target = self.transform_target(target)
            target = transform_target
        
        img = to_tensor(img)
        target = torch.as_tensor(target)

        return img, target

In [ ]:
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=32,
                                           shuffle=True,
                                           )

# 4. Pretrained Model

In [ ]:
# Download and load the pretrained ResNet-18
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetuning only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False # torch.tensor의 requires_grad attr이 False가 되면서 업데이트를 하지 않게 됩니다.

# Replace the top layer for finetuning
resnet.fc = nn.Linear(resnet.fc.in_features, 100) 
# 최상단의 레이어는 fully connected layer인데 100은 그냥 넣어본 숫자이고 output으로 나갈 수 있게끔 차원을 조정하면 됩니다.

# Forward pass.
images = torch.randn(64,3,224, 224)
outputs = resnet(images)
print(outputs.size())

# 5. Save and load the model

In [ ]:
# Save and load the entire model.
## Save
torch.save(resnet, 'model.ckpt')
## Load
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended)
## Save
torch.save(resnet.state_dict(), 'params.ckpt')

## Load
resnet.load_state_dict(torch.load('params.ckpt'))